In [1]:
import json
from sim import *
import functools
from copy import deepcopy

In [2]:

class OpsConverter:
    def __init__(self, data, needs_gpu):
        unique_ops = list({entry['name'] for entry in data})
        unique_ops.sort()
        self.ops = [(i,op,needs_gpu(op)) for i,op in enumerate(unique_ops)]
        self.count = len(self.ops)
        self.id_to_ops = {i:op for i,op,_ in self.ops}
        self.ops_to_id = {op:i for i,op,_ in self.ops}
        self.ids_to_needs_gpu = {i: needs_gpu for i,_,needs_gpu in self.ops}
        self.dep_matrix = [[False]*self.count]*self.count
    def __repr__(self):
        return str(self.ops)
    def get_all_ops(self):
        return self.ops
    def get_id(self,name):
        return self.ops_to_id[name]
    def get_op(self,idx):
        return self.id_to_ops[id]
    def get_gpu(self,name):
        return self.ids_to_needs_gpu[self.get_id(name)]
    def get_deps(self):
        return self.dep_matrix
    def set_deps(self,from_name, to_name):
        self.dep_matrix[from_name][to_name] = True

class EventDataloader:
    def __init__(self,path, verbose = 0):
        with open(path,) as f:
            data = json.load(f)
        start_time = functools.reduce( lambda x, y: x if x['ts'] < y['ts'] else y, data)['ts']
        for entry in data:
            ts = 0
            if entry['args']['timestamp'] != '[]': 
                ts = int(entry['args']['timestamp'][1:-1])
            entry['args']['timestamp'] = ts 
            entry['ts'] -= start_time
        self.data = data
        if verbose > 0:
            print (f"Times are relative to UNIX time: {start_time}; \n first event in replay starts at {data[0]['ts']} and end at {data[-1]['ts']}")
            print (f"There are {len(data)} events.")
        ids = list({ entry['args']['timestamp'] for entry in data })
        ids.sort()
        self.ids = ids
        if verbose > 0:
            print (f" There are {len(ids)} logical time labels ending with {ids[-1]}")
        self.ops_converter = OpsConverter(data, lambda name: True if ('rcnn' in name or 'lanenet' in name) else False)
        self.event_from_log = [ 
             Event(idx,
                   self.ops_converter.get_id(entry['name']), 
                   entry['ts'],
                   actual_runtime = entry['dur'], 
                   deadline=None, 
                   needs_gpu=self.ops_converter.get_gpu(entry['name'])
                  ) 
             for idx, entry in enumerate(data)
        ]
        self.lattice = Lattice([Operator(idx, [], 10, needs_gpu = needs_gpu , name = name) 
                                for idx,name,needs_gpu in self.ops_converter.get_all_ops()])
    def get_data(self):
        return self.data
    def get_data_as_binned(self):
        # prolly use this to build dependency matrix; for now online computed cause it's expensive
        return [ [ entry for entry in self.data if entry['args']['timestamp'] == ts] for ts in self.ids]
    def get_ops_converter(self):
        return self.ops_converter
    def get_event_list(self):
        return self.event_from_log
    def get_lattice(self):
        return self.lattice
    

In [3]:
events = EventDataloader("pylot_profile.json", verbose = 1)

Times are relative to UNIX time: 1625637636138946; 
 first event in replay starts at 0 and end at 439448724
There are 18459 events.
 There are 924 logical time labels ending with 52961


In [4]:

events.get_ops_converter()

[(0, 'behavior_planning.on_watermark', False), (1, 'center_camera_location_finder_history_operator.on_watermark', False), (2, 'center_camera_operator.process_images', False), (3, 'center_camera_operator.release_data', False), (4, 'faster-rcnn.on_msg_camera_stream', True), (5, 'front_center_lidar_operator.process_point_clouds', False), (6, 'gnss_operator.process_gnss', False), (7, 'imu_operator.process_imu', False), (8, 'lanenet_lane_detection.on_camera_frame', True), (9, 'linear_prediction_operator.generate_predicted_trajectories', False), (10, 'localization_operator.on_watermark', False), (11, 'pid_control_operator.on_watermark', False), (12, 'planning_operator.on_prediction_update', False), (13, 'planning_operator.on_time_to_decision', False), (14, 'planning_operator.on_watermark', False), (15, 'simulator_bridge_operator.on_control_msg', False), (16, 'simulator_bridge_operator.send_actor_data', False), (17, 'tracker_sort.on_watermark', False), (18, 'traffic_light_camera_location_find

In [5]:
b = [(entry['name'], entry['args'], entry['ts']) for entry in events.get_data_as_binned()[-2]]

In [6]:
b

[('gnss_operator.process_gnss', {'timestamp': 52911}, 438543587),
 ('front_center_lidar_operator.process_point_clouds',
  {'timestamp': 52911},
  438557720),
 ('simulator_bridge_operator.send_actor_data',
  {'timestamp': 52911},
  438539467),
 ('imu_operator.process_imu', {'timestamp': 52911}, 438587383),
 ('localization_operator.on_watermark', {'timestamp': 52911}, 438588697),
 ('behavior_planning.on_watermark', {'timestamp': 52911}, 438591248),
 ('planning_operator.on_time_to_decision', {'timestamp': 52911}, 438591830),
 ('traffic_light_camera_operator.process_images',
  {'timestamp': 52911},
  438751770),
 ('center_camera_operator.process_images', {'timestamp': 52911}, 438754872),
 ('faster-rcnn.on_msg_camera_stream', {'timestamp': 52911}, 438796827),
 ('tracker_sort.on_watermark', {'timestamp': 52911}, 438877905),
 ('traffic_light_detector_operator.on_frame', {'timestamp': 52911}, 438786973),
 ('traffic_light_camera_location_finder_operator.on_watermark',
  {'timestamp': 52911},
  

In [7]:
events.get_data()[0]

{'name': 'center_camera_operator.release_data',
 'pid': 'center_camera_operator',
 'tid': 1,
 'ts': 0,
 'dur': 39,
 'ph': 'X',
 'args': {'timestamp': 0}}

In [8]:
events.get_event_list()[0:5]

[<Event 0; Running Op 3; Available at Time 0; Executed None to None; Deadline: None>,
 <Event 1; Running Op 20; Available at Time 886; Executed None to None; Deadline: None>,
 <Event 2; Running Op 6; Available at Time 4009432; Executed None to None; Deadline: None>,
 <Event 3; Running Op 5; Available at Time 4026421; Executed None to None; Deadline: None>,
 <Event 4; Running Op 7; Available at Time 4052420; Executed None to None; Deadline: None>]

In [9]:
events.get_lattice()

Lattice {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: []}

In [10]:
class Emulator: 
    def __init__(self):
        self.assignment = [] # list of worker_id X task_id 
        self.release = 0 # time when assignment is made available
    def set_schedule (schedule,task_set,worker_pool,dependencies,timeout,v=0):
        """
            schedule -- scheduling function
            task_set -- list of tasks
            worker_pool -- workers to be given events
            dependencies -- matrix of dependencies between tasks
        """
        start = time.time()
        self.assignments, self.release = schedule(task_set, worker_pool, dependencies, timeout)
        end = time.time()
    def get_schedule(curr_time):
        if curr_time >= self.release:
            return self.assignment
        else:
            return None
    def reset():
        self.__init__()

class Simulator:
    def __init__( self, num_cpu, num_gpu, events_list, lattice):
        '''
            num_cpu -- number of cpus
            num_gpu -- number of gpus
            events_list -- list of Events
            lattice -- lattice :: default one would just have operators not connected to each other
        '''
        self.events_list = events_list.copy()
        self.worker_pool = WorkerPool([Worker(i, gpus=(1 if i < num_gpu else 0)) for i in range(num_cpu + num_gpu)])
        self.lattice = deepcopy(lattice)
        self.emulator = Emulator()
        self.current_time = 0
        self.current_events = []
        
    def simulate(self,timeout, replay_log, deps):
        # set up 
        for current_time in range(timeout):
            self.step()
            
    def step(self): 
        # first determine if there's new tasks to be made visible
        # second determine where to place tasks
        # finally advance the workers and time
        pass
    def simulate(self,schedule, timeout,v=0):
        if not self.events_list:
            return
        self.events_list.sort(key=(lambda e: e.arrival_time))
        event_queue = []
        for time in range(timeout):
            if v:
                print ("step: {}".format(time))
            while len(self.events_list) != 0 and  self.events_list[0].arrival_time == time:
                task = self.events_list.pop(0)
                print ("Activate: {}".format(task))
                event_queue.append(task)
            schedule(time,event_queue,self.lattice,self.worker_pool,timeout)

In [11]:
s = Simulator(2,1,events.get_event_list()[0:10], events.get_lattice())

In [16]:
s.simulate(fifo_schedule, 4228295 + 100)

Activate: <Event 2; Running Op 6; Available at Time 4009432; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 6; Available at Time 4009432; Executed 4009432 to 4010916; Deadline: None>
Activate: <Event 3; Running Op 5; Available at Time 4026421; Executed None to None; Deadline: None>
finished event: <Event 3; Running Op 5; Available at Time 4026421; Executed 4026421 to 4030068; Deadline: None>
Activate: <Event 4; Running Op 7; Available at Time 4052420; Executed None to None; Deadline: None>
finished event: <Event 4; Running Op 7; Available at Time 4052420; Executed 4052420 to 4053894; Deadline: None>
Activate: <Event 7; Running Op 16; Available at Time 4055579; Executed None to None; Deadline: None>
Activate: <Event 5; Running Op 10; Available at Time 4058387; Executed None to None; Deadline: None>
finished event: <Event 5; Running Op 10; Available at Time 4058387; Executed 4058387 to 4059417; Deadline: None>
Activate: <Event 8; Running Op 0; Available at Ti

In [13]:
l = events.get_lattice()

In [14]:
deepcopy(l)

Lattice {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: []}

In [15]:
s.events_list

[<Event 2; Running Op 6; Available at Time 4009432; Executed None to None; Deadline: None>,
 <Event 3; Running Op 5; Available at Time 4026421; Executed None to None; Deadline: None>,
 <Event 4; Running Op 7; Available at Time 4052420; Executed None to None; Deadline: None>,
 <Event 7; Running Op 16; Available at Time 4055579; Executed None to None; Deadline: None>,
 <Event 5; Running Op 10; Available at Time 4058387; Executed None to None; Deadline: None>,
 <Event 8; Running Op 0; Available at Time 4060117; Executed None to None; Deadline: None>,
 <Event 6; Running Op 13; Available at Time 4062088; Executed None to None; Deadline: None>,
 <Event 9; Running Op 19; Available at Time 4228295; Executed None to None; Deadline: None>]